# Web Scraping in Python for Beginners

In this notebook, I will give an introduction to webscraping in python. This will not be an exhaustive overview of all the resources at your disposal with python, but it should be enough to help you in the beginning. The library I will be going over in this post is Beautiful Soup (utilizing Requests). In my next tutorial, I will go over how to utalize Beautiful Soup and Selenium together. The only other major option you have available is Scrapy. Along the way, I will discuss your options and leave more tutorials for different aspects of this project that are outside of the scope of Beautiful Soup.

#### First we need to import the needed libraries: 

In [8]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd
import re
import urllib

### Beautiful Soup _Only_:

Beautiful Soup is a python package that parces XML and HTML. It makes it very easy to get data from the html data avaible on the website. However, it does have it's pitfalls. These include not being able to grab data as easily from dynamic pages. In those situations we will use Selenium.

So, in the cell below we can start with a quick example. Let's try with a few pages to get a feel for how things change. So, below lets define three variables (url_1, url_2, and url_3) to three different urls.

The first is Zara's dresses page, flatiron data science page, and pelton's yelp page.

In [2]:
url_1 = 'https://www.zara.com/us/en/woman-dresses-l1066.html?v1=1180427'
url_2 = 'https://flatironschool.com/career-courses/data-science-bootcamp/'
url_3 = 'https://www.yelp.com/biz/peloton-new-york?osq=peloton'

So, first we need to use the Requests library to request the url and assign that to the variable page. We can then use page in the beautiful soup function, and pass 'lxml' as the second argument. This just means that Beautiful Soup is parcing the page as lxml.

By convention we will save the results of BeautifulSoup as soup (in this case FI_soup).

In [3]:
zara_page = requests.get(url_1)
zara_soup = BeautifulSoup(zara_page.content, 'lxml')

You can already view this, simply run the cell below to view the page html: 

In [4]:
zara_soup #this may time out due to the page being too big

zara_soup.find('div') #this will return the first div on the page

<div class="_global-loader active" id="global-loader"></div>

So, now that we have the html saved as a variable we can parce through the page to get the information we need.

In the pervious cell we saw one of the most common ways to grab information using Beautiful Soup. The two methods __find__ and __findAll__ will be your best friends in this process. However, there are many other ways to grab infomation from the html if these do not work. And we will discuss those next.

If we go and inspect our page, we can see the classnames and IDs of items that we want. below is the screen shot of what the it looked like when I inspected each product item. In this exercise, we will be grabbing the products name and price, and saving it as a dictionary.

<img style="float: center;" src="images/inspect_zara.png"  width=800>

if you look at it we can easily see the classname for the divs surrounding the product name and the price, **'name _item'** and **'price _product_price'** respectively.

#### Lets first see if we can grab all of the product names.

In Beautiful Soup the method findAll allows us to do just that. If we use the find method, it will only retun the first itme that it finds that meets the requirements passed, while finaAll returns every item that meets the requirements.

As seen above, the first thing we pass in the method is the html tag (i.e. 'div','p','h1','a') and the second will be a dictionary with a key value pair. Typically you will see **'class'** with the actual class name or **'ID'**. We will be using class.

After grabbing all of the links with this class name _'name'_ we will use a [list comprehension](https://hackernoon.com/list-comprehension-in-python-8895a785550b) to create a list of the names. If you are not familiar with list comprehensions, I highly recommend them! 

After, lets view the first 5 elements of the list:

In [5]:
names = zara_soup.findAll('a',{'class','name'})
product_names = [item.text for item in names]
product_names[:5]

['DRESS WITH DRAPED NECKLINE',
 'POLKA DOT DRESS',
 'LINEN DRESS WITH BUTTONS',
 'FRINGED TEXTURED WEAVE TUNIC',
 'RUFFLED DRESS']

The next part for prices gets a little complicated. Due to how the html is parced we need to use a little regex to grab the prices! So first, what we need to do is to grab the divs with the prices and lets look at the first item in the list before we move forward:

In [10]:
prices = zara_soup.findAll('div',{'class','_product-price'})
prices[0]

<div class="price _product-price"><span data-price="49.90 USD"></span></div>

Very frequently we can just use the dot method text, which would return the text in the divs. However, usually if the text is surrounded by a span we are unable to do this. If we test it below we can see that this returns an empty string:

In [11]:
prices[0].text

''

Now for some Regex. We will use Python's 'Re' library's search method as well as it's group. If we look at all of the prices we realize they are surrounded in a span and have a special html tag called 'data-price'. So to grab this information pass the first chuck of html before the price <img style="display: inline-block;" src="images/span_img.png"  width=150> and before each special character we will use a '\' to break and let the regex parcer know that we need that special charater in our search. next we will have parentheses with '.*' in it which just means any length/combination of characters, and then finish it with ' USD"><' this is how our sting should end.

In [7]:
prod_prices = [float(re.search('\<span data\-price\=\"(.*)\ USD\"\>\<',str(item)).group(1)) for item in prices]
prod_prices[:5]

[49.9, 69.9, 69.9, 49.9, 49.9]

#### The majority of your projects will consist of these steps.

However, as i mentioned before there are other ways to grab 